In [28]:
WEATHER_API_KEY='52c94fc20a582a91a0994782b8ca2acc'

In [29]:
'https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={API key}'

'https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={API key}'

In [30]:
import requests
import pandas as pd
import numpy as np

In [31]:
long=23.545447
lat=87.2876081
url = f'https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={long}&appid={WEATHER_API_KEY}'

In [32]:
response = requests.get(url)

In [33]:
wind_speed = response.json()['wind']['speed']
weather = response.json()['weather'][0]
sunset_time = response.json()['sys']['sunset']
sunrise_time = response.json()['sys']['sunrise']
cloud_cover = response.json()['clouds']['all']

Insolation:
$Q = S (\frac{r}{r_0})^2 \cos θ_0$

Q is the daytime insolation (in watts per square meter)  
S is the solar constant (1361 watts per square meter)  
r is the Earth's distance from the Sun (in meters)  
$r_0$ is the Earth's mean distance from the Sun (149.6 million meters)  
$θ_0$ is the solar zenith angle (the angle between the sun's rays and the vertical)  

---

$r = r_0 (1 + 0.0167 \cos(360 * \frac{d-3}{365}))$  
d is the number of days since January 1st

$θ_0 = \arccos( \sin (latitude) * \sin (\delta) + \cos(latitude) * \cos (\delta)  * \cos (hour angle) )$

$hourangle = (time of day in hours - 12) * 15°$

$\delta = 23.45° * \sin(360° * \frac{N - 81}{365})$  
N is the day of the year (1-365)

---

Strong	    > 100  
Moderate	50-100  
Slight	    < 50  

In [34]:
from datetime import date, datetime

today = date.today()

In [35]:
N = pd.Period(today, freq='D').dayofyear
delta = 23.45 * np.sin(np.deg2rad(360 * (N-81) / 365))
timeofdayinhrs = datetime.now().hour
hourangle = (timeofdayinhrs - 12) * 15
theta_not = np.arccos(np.sin(lat) * np.sin(delta) + np.cos(lat) * np.cos(delta) * np.cos(hourangle))
d = (date.today() - date(date.today().year,1,1)).days
r_by_r0 = (1 + 0.0167*np.cos(360*(d-3)/365))

In [36]:
Q = 1361 * r_by_r0**2 * np.cos(theta_not)

In [37]:
daytime_insolation_class = ''

if Q > 100:
    daytime_insolation_class = 'strong'
elif Q > 50 and Q <= 100:
    daytime_insolation_class = 'moderate'
else:
    daytime_insolation_class = 'slight'

In [38]:
Q, daytime_insolation_class

(923.3489374574697, 'strong')

Nighttime Conditions

In [39]:
nighttime_cond = ''
if cloud_cover >= 80:
    nighttime_cond = 'overcast'
else:
    nighttime_cond = 'not overcast'

In [40]:
cloud_cover, nighttime_cond

(99, 'overcast')

---

In [44]:
datetime.utcfromtimestamp(sunrise_time).hour

3

In [46]:
if datetime.now().hour >= datetime.utcfromtimestamp(sunrise_time).hour + 1 and datetime.now().hour <= datetime.utcfromtimestamp(sunset_time).hour - 1:
    if wind_speed < 2:
        if daytime_insolation_class == 'strong':
            stability_class = 'A'
        elif daytime_insolation_class == 'moderate':
            stability_class = 'B'
        else:
            stability_class = 'B'
    if wind_speed >= 2 and wind_speed < 3:
        if daytime_insolation_class == 'strong':
            stability_class = 'B'
        elif daytime_insolation_class == 'moderate':
            stability_class = 'B'
        else:
            stability_class = 'C'
    if wind_speed >= 3 and wind_speed < 4:
        if daytime_insolation_class == 'strong':
            stability_class = 'B'
        elif daytime_insolation_class == 'moderate':
            stability_class = 'C'
        else:
            stability_class = 'C'
    if wind_speed >= 4 and wind_speed < 6:
        if daytime_insolation_class == 'strong':
            stability_class = 'C'
        elif daytime_insolation_class == 'moderate':
            stability_class = 'D'
        else:
            stability_class = 'D'
    if wind_speed >= 6:
        if daytime_insolation_class == 'strong':
            stability_class = 'C'
        elif daytime_insolation_class == 'moderate':
            stability_class = 'D'
        else:
            stability_class = 'D'
else:
    if wind_speed < 2:
        stability_class = 'F'
    if wind_speed >= 2 and wind_speed < 3:
        if nighttime_cond == 'overcast':
            stability_class = 'F'
        else:
            stability_class = 'E'
    if wind_speed >= 3 and wind_speed < 4:
        if nighttime_cond == 'overcast':
            stability_class = 'E'
        else:
            stability_class = 'D'
    if wind_speed >= 4 and wind_speed < 6:
        stability_class = 'D'
    if wind_speed >= 6:
        stability_class = 'D'

In [47]:
stability_class

'C'

determining r

In [ ]:
E_tnt = 4686 # kJ/kg
m = 1 # kg => needed to be determined from plant data (may be database or sensor)
eta = 0.15 # explosion efficiency => needed to be estimated but took the worst case where the range would be max.
delHc = 0
po = 0 # peak sideon overpressure from sensor measurement / database 
pa = 0 # ambient pressure from sensor measurement / database
ps = po/pa

In [ ]:
m_tnt = (eta*m*delHc)/E_tnt
ze = (ps/16.2499)**(-1/1.8093) # m/kg^(1/3)
r = ze*(m_tnt**(1/3))

Puff model

In [ ]:
sigma_x = 0
sigma_y = 0
sigma_z = 0

if stability_class == 'A':
    